<a href="https://colab.research.google.com/github/JustG-ui/Redes-Neurais-Convolucionais/blob/main/2025_1_Treinamento_de_modelo_CNN_para_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
# Nomes dos alunos: Henrique Vitral, Henrique Costa Gomes, Lucas Seabra, João Pedro Boanerges

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np

# Configuração de dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

## 1. Download e Preparação do Dataset MNIST
def carregar_dados():
    """
    Carrega os datasets MNIST de treino e teste.

    Retorna:
        train_loader: DataLoader para dados de treino
        test_loader: DataLoader para dados de teste
    """
    # Transformações aplicadas nas imagens
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])

    # Download dos datasets
    train_dataset = datasets.MNIST(
        root='./data',
        train=True,
        download=True,
        transform=transform
    )

    test_dataset = datasets.MNIST(
        root='./data',
        train=False,
        download=True,
        transform=transform
    )

    # Cria os DataLoaders para carregar os dados em batches
    train_loader = DataLoader(
        train_dataset,
        batch_size=64,
        shuffle=True,
        num_workers=2
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=1000,
        shuffle=False,
        num_workers=2
    )

    return train_loader, test_loader


Usando dispositivo: cpu


In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## 2. Definição da Arquitetura CNN
class CNN_MNIST(nn.Module):
    """
    Rede Neural Convolucional para classificação do dataset MNIST.

    Arquitetura:
    - Conv2d (1->32) + ReLU + Conv2d (32->64) + ReLU + MaxPool + Dropout
    - Conv2d (64->128) + ReLU + MaxPool + Dropout
    - Flatten + Linear (128*3*3->256) + ReLU + Dropout + Linear (256->10)
    """

    def __init__(self):
        super(CNN_MNIST, self).__init__()

        # Primeira camada convolucional
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)

        # Segunda camada convolucional
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)

        # Camadas totalmente conectadas
        self.fc1 = nn.Linear(128 * 7 * 7, 256)
        self.fc2 = nn.Linear(256, 10)

        # Dropout para regularização
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)

    def forward(self, x):
        # Primeira sequência convolucional
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)

        # Segunda sequência convolucional
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)

        # Flatten e camadas totalmente conectadas
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)

        return F.log_softmax(x, dim=1)

In [22]:
## 3. Funções de Treino e Teste

import torch
import torch.nn as nn

def treinar(modelo, dispositivo, train_loader, optimizer, epoch):
    """
    Executa uma época de treinamento do modelo.

    Args:
        modelo: Modelo CNN a ser treinado
        dispositivo: Dispositivo onde os cálculos serão feitos (CPU/GPU)
        train_loader: DataLoader com dados de treino
        optimizer: Otimizador para ajuste dos pesos
        epoch: Número da época atual

    Retorna:
        loss_medio: Loss médio da época
        acuracia: Acurácia do modelo nos dados de treino
    """
    modelo.train()  # Modo treino
    loss_total = 0
    corretos = 0

    for batch_idx, (dados, alvos) in enumerate(train_loader):
        dados, alvos = dados.to(dispositivo), alvos.to(dispositivo)

        # Zera os gradientes
        optimizer.zero_grad()

        # Forward pass
        output = modelo(dados)

        # Calcula o loss
        loss = nn.functional.nll_loss(output, alvos)

        # Backward pass
        loss.backward()

        # Atualiza os pesos
        optimizer.step()

        # Acumula estatísticas
        loss_total += loss.item()
        pred = output.argmax(dim=1, keepdim=True)
        corretos += pred.eq(alvos.view_as(pred)).sum().item()

    # Calcula médias
    tamanho_dataset = len(train_loader.dataset)
    loss_medio = loss_total / len(train_loader)
    acuracia = 100. * corretos / tamanho_dataset

    print(f"Treino - Época {epoch}: Loss médio: {loss_medio:.4f}, Acurácia: {corretos}/{tamanho_dataset} ({acuracia:.2f}%)")

    return loss_medio, acuracia

def testar(modelo, dispositivo, test_loader):
    """
    Avalia o modelo nos dados de teste.

    Args:
        modelo: Modelo CNN a ser avaliado
        dispositivo: Dispositivo onde os cálculos serão feitos (CPU/GPU)
        test_loader: DataLoader com dados de teste

    Retorna:
        loss_medio: Loss médio no conjunto de teste
        acuracia: Acurácia do modelo nos dados de teste
    """
    modelo.eval()
    loss_total = 0
    corretos = 0

    with torch.no_grad():
        for dados, alvos in test_loader:
            dados, alvos = dados.to(dispositivo), alvos.to(dispositivo)

            # Forward pass
            output = modelo(dados)

            # Calcula o loss
            loss_total += nn.functional.nll_loss(output, alvos, reduction='sum').item()

            # Calcula acertos
            pred = output.argmax(dim=1, keepdim=True)
            corretos += pred.eq(alvos.view_as(pred)).sum().item()

    # Calcula médias
    tamanho_dataset = len(test_loader.dataset)
    loss_medio = loss_total / tamanho_dataset
    acuracia = 100. * corretos / tamanho_dataset

    print(f"Teste - Loss médio: {loss_medio:.4f}, Acurácia: {corretos}/{tamanho_dataset} ({acuracia:.2f}%)")

    return loss_medio, acuracia

In [24]:
## 4. Treinamento com Early Stopping

import torch
import torch.optim as optim

def treinamento_com_early_stopping(modelo, dispositivo, train_loader, test_loader, max_epocas=20):
    """
    Executa o treinamento do modelo com early stopping para evitar overfitting.

    Args:
        modelo: Modelo CNN a ser treinado
        dispositivo: Dispositivo onde os cálculos serão feitos (CPU/GPU)
        train_loader: DataLoader com dados de treino
        test_loader: DataLoader com dados de teste
        max_epocas: Número máximo de épocas de treino

    Retorna:
        historico: Dicionário com histórico de loss e acurácia
    """
    optimizer = optim.Adam(modelo.parameters(), lr=0.0001)
    historico = {
        'train_loss': [],
        'train_acc': [],
        'test_loss': [],
        'test_acc': [],
        'melhor_loss': float('inf'),
        'epocas_sem_melhoria': 0,
        'melhor_epoca': 0
    }

    for epoch in range(1, max_epocas + 1):
        # Executa uma época de treino
        train_loss, train_acc = treinar(modelo, dispositivo, train_loader, optimizer, epoch)

        # Avalia no conjunto de teste
        test_loss, test_acc = testar(modelo, dispositivo, test_loader)

        # Armazena no histórico
        historico['train_loss'].append(train_loss)
        historico['train_acc'].append(train_acc)
        historico['test_loss'].append(test_loss)
        historico['test_acc'].append(test_acc)

        # Verifica early stopping
        if test_loss < historico['melhor_loss']:
            historico['melhor_loss'] = test_loss
            historico['epocas_sem_melhoria'] = 0
            historico['melhor_epoca'] = epoch
            # Salva o melhor modelo
            torch.save(modelo.state_dict(), 'melhor_modelo_mnist.pt')
        else:
            historico['epocas_sem_melhoria'] += 1
            if historico["epocas_sem_melhoria"] >= 3:
                print(f"\nEarly stopping na época {epoch}! O erro de teste não melhorou desde a época {historico['melhor_epoca']}.")
                break

    # Carrega o melhor modelo antes do overfitting
    modelo.load_state_dict(torch.load('melhor_modelo_mnist.pt'))

    return historico


In [23]:
## 5. Visualização dos Resultados

import matplotlib.pyplot as plt

def plotar_resultados(historico):
    """
    Plota gráficos de loss e acurácia durante o treinamento.

    Args:
        historico: Dicionário com histórico de treinamento
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

    # Gráfico de Loss
    ax1.plot(historico['train_loss'], label='Treino')
    ax1.plot(historico['test_loss'], label='Teste')
    ax1.set_title('Loss durante o Treinamento')
    ax1.set_xlabel('Época')
    ax1.set_ylabel('Loss')
    ax1.legend()

    # Gráfico de Acurácia
    ax2.plot(historico['train_acc'], label='Treino')
    ax2.plot(historico['test_acc'], label='Teste')
    ax2.set_title('Acurácia durante o Treinamento')
    ax2.set_xlabel('Época')
    ax2.set_ylabel('Acurácia (%)')
    ax2.legend()

    plt.tight_layout()
    plt.show()


In [ ]:
import torch
import matplotlib.pyplot as plt

# 6. Exemplo de Uso do Modelo Treinado
def exemplo_uso(modelo, dispositivo, test_loader):
    """
    Mostra um exemplo de classificação usando o modelo treinado.

    Args:
        modelo: Modelo CNN treinado
        dispositivo: Dispositivo onde os cálculos serão feitos (CPU/GPU)
        test_loader: DataLoader com dados de teste
    """
    # Pega um batch de exemplos de teste
    data_iter = iter(test_loader)
    dados, alvos = next(data_iter)

    # Seleciona apenas 9 exemplos para visualização
    dados = dados[:9]
    alvos = alvos[:9]

    dados, alvos = dados.to(dispositivo), alvos.to(dispositivo)

    # Faz a predição
    modelo.eval()
    with torch.no_grad():
        output = modelo(dados)

    # Pega as predições
    preds = output.argmax(dim=1)

    # Mostra os exemplos
    fig, axes = plt.subplots(3, 3, figsize=(9, 9))
    for i, ax in enumerate(axes.flat):
        img = dados[i].cpu().numpy().squeeze()
        ax.imshow(img, cmap='gray')
        ax.set_title(f"Predição: {preds[i].item()}\nReal: {alvos[i].item()}")
        ax.axis('off')
    plt.tight_layout()
    plt.show()


# 7. Fluxo Principal
def main():
    # Carrega os dados
    train_loader, test_loader = carregar_dados()

    # Cria o modelo e envia para o dispositivo (GPU/CPU)
    modelo = CNN_MNIST().to(device)

    print("\nIniciando treinamento...")
    print("--------------------------------------------------")

    # Executa o treinamento
    historico = treinamento_com_early_stopping(
        modelo,
        device,
        train_loader,
        test_loader,
        max_epocas=20
    )

    # Plota os resultados
    plotar_resultados(historico)

    # Mostra um exemplo de uso
    print("\nExemplos de classificação:")
    exemplo_uso(modelo, device, test_loader)

if __name__ == '__main__':
    main()


Iniciando treinamento...
--------------------------------------------------
Treino - Época 1: Loss médio: 0.3909, Acurácia: 52714/60000 (87.86%)
Teste - Loss médio: 0.0827, Acurácia: 9735/10000 (97.35%)
Treino - Época 2: Loss médio: 0.1109, Acurácia: 58001/60000 (96.67%)
Teste - Loss médio: 0.0520, Acurácia: 9829/10000 (98.29%)
Treino - Época 3: Loss médio: 0.0778, Acurácia: 58568/60000 (97.61%)
Teste - Loss médio: 0.0362, Acurácia: 9882/10000 (98.82%)
Treino - Época 4: Loss médio: 0.0624, Acurácia: 58879/60000 (98.13%)
Teste - Loss médio: 0.0314, Acurácia: 9895/10000 (98.95%)
Treino - Época 5: Loss médio: 0.0516, Acurácia: 59029/60000 (98.38%)
Teste - Loss médio: 0.0266, Acurácia: 9903/10000 (99.03%)


KeyboardInterrupt: 